In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split,KFold

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sub=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')
tr=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
ts=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')
print(tr.shape,ts.shape)

In [ ]:
#  target encoding
#  encoding the cat features by means of target on 5 fold
#  use the encoder for test

In [ ]:
cat_fe=[c for c in tr.columns if tr[c].dtypes=='O']
kf=KFold(n_splits=5,random_state=1)
for c in cat_fe:
    tr[c+'_TE']=None
    ts[c+'_TE']=0
for train_index, test_index in kf.split(tr):
    for col in cat_fe:
        dic=tr.groupby(col).agg({'target':'mean'}).to_dict()['target']
        tr.loc[test_index,col+'_TE']=tr.loc[test_index,col].map(dic).values
        ts[col+'_TE']+=ts[col].map(dic).values

for c in cat_fe:
    ts[c+'_TE']/=5

In [ ]:
# Ridge
for i in np.arange(0,1000,100):
    clf=Ridge(i)
    clf.fit(tr[features].values,tr.target)
    print(pow(mse(tr.target,clf.predict(tr[features])),.5))

In [ ]:
# decision tree
from sklearn import tree
clf1=tree.DecisionTreeRegressor(max_depth=6)
clf1.fit(tr[features].values,tr.target)
pow(mse(tr.target,clf1.predict(tr[features])),.5)

In [ ]:
# best depth for a single tree?

X,x,Y,y=train_test_split(tr[features].values,tr.target,train_size=0.7,random_state=1)
score_res=[]
for i in np.arange(1,14,1):
    clf1=tree.DecisionTreeRegressor(max_depth=i)
    clf1.fit(X,Y)
    score_res.append(pow(mse(y,clf1.predict(x)),.5))
sns.relplot(np.arange(1,14,1),score_res)

In [ ]:
cont_cols=[c for c in tr.columns if 'cont' in c]

In [ ]:
X,x,Y,y=train_test_split(tr[cont_cols].values,tr.target,train_size=0.7,random_state=1)
score_res=[]
for i in np.arange(1,14,1):
    clf1=tree.DecisionTreeRegressor(max_depth=i)
    clf1.fit(X,Y)
    score_res.append(pow(mse(y,clf1.predict(x)),.5))
sns.relplot(np.arange(1,14,1),score_res)

In [ ]:

tr_dummy=pd.get_dummies(tr)
all_col=[c for c in tr_dummy.columns if c != 'target']
X,x,Y,y=train_test_split(tr_dummy[all_col].values,tr.target,train_size=0.7,random_state=1)
score_res=[]
for i in np.arange(1,14,1):
    clf1=tree.DecisionTreeRegressor(max_depth=i)
    clf1.fit(X,Y)
    score_res.append(pow(mse(y,clf1.predict(x)),.5))
sns.relplot(np.arange(1,14,1),score_res)